In [ ]:
# Ch15-4 NextFlow

In [ ]:
## NextFlow example with FastQC analysis ##

In [ ]:
# 1. Import Libraries
import os
import sys
import subprocess
import time
import threading
import queue
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import signal
import psutil

In [ ]:
# 2. FastQC Pipeline Class
class SimpleFastQCPipeline:
    """Simplified Nextflow pipeline using system FastQC"""
    
    def __init__(self):
        self.workflow_file = "main.nf"
        self.process = None
        
    def setup_pipeline(self):
        """Setup the complete pipeline"""
        print("🔧 Setting up simplified Nextflow pipeline...")
        
        # Create directories
        dirs = ["data", "reference", "results", "reports"]
        for d in dirs:
            Path(d).mkdir(parents=True, exist_ok=True)
        
        # Create the Nextflow workflow file
        self._create_nextflow_workflow()
        
        # Create configuration
        self._create_nextflow_config()
        
        # Create sample data
        self._create_sample_data()
        
        # Create reference files
        self._create_reference_files()
        
        # Check FastQC installation
        self._check_fastqc_installation()
        
        print("✅ Pipeline setup complete!")
        
    def _create_nextflow_workflow(self):
        """Create simplified Nextflow workflow"""
        workflow_content = '''#!/usr/bin/env nextflow

nextflow.enable.dsl=2

// Parameters
params.reads = "data/*_{R1,R2}.fastq.gz"
params.reference = "reference/genome.fa"
params.outdir = "results"
params.test_mode = false

log.info """
==============================================
Simplified FastQC Pipeline
==============================================
reads        : ${params.reads}
reference    : ${params.reference}
outdir       : ${params.outdir}
test_mode    : ${params.test_mode}
==============================================
"""

// Main workflow
workflow {
    // Create input channels
    reads_ch = Channel.fromFilePairs(params.reads, checkIfExists: true)
    
    // Run FastQC
    fastqc(reads_ch)
    
    // Trim reads (simplified)
    trimmomatic(reads_ch)
    
    // Generate final report
    multiqc(fastqc.out.zip.collect())
}

// FastQC process
process fastqc {
    tag "$sample_id"
    publishDir "${params.outdir}/fastqc", mode: 'copy'
    
    input:
    tuple val(sample_id), path(reads)
    
    output:
    path "*.html", emit: html
    path "*.zip", emit: zip
    
    script:
    if (params.test_mode)
        """
        echo "Running FastQC in TEST MODE for $sample_id"
        echo "Input files: $reads"
        
        # Create realistic mock FastQC outputs
        for read in $reads; do
            base=\$(basename \$read .fastq.gz)
            
            cat << EOF > \${base}_fastqc.html
<!DOCTYPE html>
<html>
<head><title>FastQC Report: \$base (Test Mode)</title></head>
<body style="font-family: Arial, sans-serif; margin: 20px;">
<h1>FastQC Report - Test Mode</h1>
<h2>Sample: \$base</h2>
<p><em>This is a test mode report</em></p>

<h3>Basic Statistics</h3>
<table border="1" style="border-collapse: collapse; margin: 10px 0;">
<tr style="background-color: #f0f0f0;"><td><b>Measure</b></td><td><b>Value</b></td></tr>
<tr><td>Total Sequences</td><td>5,000</td></tr>
<tr><td>Sequence length</td><td>100</td></tr>
<tr><td>%GC</td><td>50</td></tr>
</table>

<h3>Quality Assessment</h3>
<ul>
<li>✅ <b>Per base sequence quality:</b> PASS</li>
<li>✅ <b>Per sequence quality scores:</b> PASS</li>
<li>✅ <b>Per base sequence content:</b> PASS</li>
<li>✅ <b>Per sequence GC content:</b> PASS</li>
<li>✅ <b>Per base N content:</b> PASS</li>
<li>✅ <b>Sequence Length Distribution:</b> PASS</li>
<li>✅ <b>Sequence Duplication Levels:</b> PASS</li>
<li>✅ <b>Overrepresented sequences:</b> PASS</li>
<li>✅ <b>Adapter Content:</b> PASS</li>
</ul>

<h3>Summary</h3>
<p>This sample passed all quality checks in test mode. For real analysis, install FastQC and run with system mode.</p>
</body>
</html>
EOF
            
            # Create mock ZIP file with more realistic content
            echo "FastQC test data for \$base generated on \$(date)" > \${base}_fastqc.zip
        done
        
        echo "✅ Test mode FastQC completed for $sample_id"
        """
    else
        """
        echo "Running system FastQC for $sample_id"
        echo "Input files: $reads"
        
        # Check if FastQC is available
        if ! command -v fastqc >/dev/null 2>&1; then
            echo "❌ FastQC not found in system PATH"
            echo "Please install FastQC first:"
            echo "  - macOS: brew install fastqc"
            echo "  - Ubuntu: sudo apt install fastqc"
            echo "  - conda: conda install -c bioconda fastqc"
            
            # Create informative mock outputs
            for read in $reads; do
                base=\$(basename \$read .fastq.gz)
                
                cat << EOF > \${base}_fastqc.html
<!DOCTYPE html>
<html>
<head><title>FastQC Installation Required</title></head>
<body style="font-family: Arial, sans-serif; margin: 20px;">
<h1>FastQC Not Found</h1>
<h2>Sample: \$base</h2>
<p><b>FastQC is not installed on your system.</b></p>

<h3>Installation Instructions:</h3>
<ul>
<li><b>macOS:</b> <code>brew install fastqc</code></li>
<li><b>Ubuntu/Debian:</b> <code>sudo apt install fastqc</code></li>
<li><b>conda:</b> <code>conda install -c bioconda fastqc</code></li>
<li><b>Manual:</b> Download from <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/">FastQC website</a></li>
</ul>

<p>After installation, run the pipeline again to get real quality analysis.</p>
</body>
</html>
EOF
                
                echo "FastQC not available for \$base" > \${base}_fastqc.zip
            done
            
            echo "⚠️  FastQC not found - created installation instructions"
            exit 0
        fi
        
        # Run real FastQC
        echo "✅ FastQC found, running analysis..."
        fastqc --version
        
        # Run FastQC with timeout protection
        timeout 1800 fastqc $reads --outdir . --threads 1 || {
            echo "❌ FastQC execution failed or timed out"
            exit 1
        }
        
        echo "✅ FastQC analysis completed for $sample_id"
        ls -la *.html *.zip
        """
}

// Simplified trimming process
process trimmomatic {
    tag "$sample_id"
    publishDir "${params.outdir}/trimmed", mode: 'copy'
    
    input:
    tuple val(sample_id), path(reads)
    
    output:
    path "*_trimmed.fastq.gz", emit: trimmed
    path "*.log", emit: log
    
    script:
    """
    echo "Simulating read trimming for $sample_id"
    
    # Create trimmed files (copy originals with new names)
    for read in $reads; do
        base=\$(basename \$read .fastq.gz)
        cp \$read \${base}_trimmed.fastq.gz
    done
    
    # Create log file
    cat << EOF > ${sample_id}_trimming.log
Input Read Pairs: 5000
Both Surviving: 4850 (97.00%)
Forward Only Surviving: 120 (2.40%)
Reverse Only Surviving: 20 (0.40%)
Dropped: 10 (0.20%)
EOF
    
    echo "✅ Read trimming completed for $sample_id"
    """
}

// Simplified MultiQC process
process multiqc {
    publishDir "${params.outdir}/multiqc", mode: 'copy'
    
    input:
    path qc_files
    
    output:
    path "multiqc_report.html"
    path "multiqc_*", optional: true
    
    script:
    """
    echo "Generating quality report..."
    
    # Try to run MultiQC if available
    if command -v multiqc >/dev/null 2>&1; then
        echo "✅ MultiQC found, generating comprehensive report..."
        timeout 300 multiqc . --filename multiqc_report.html --force || {
            echo "MultiQC timed out, creating basic report..."
        }
        
        # Handle MultiQC data directory
        if [ -d "multiqc_report_data" ]; then
            mv multiqc_report_data multiqc_data
        fi
    else
        echo "MultiQC not found, creating basic summary report..."
    fi
    
    # Always create a report (fallback if MultiQC fails)
    if [ ! -f "multiqc_report.html" ]; then
        cat << 'EOF' > multiqc_report.html
<!DOCTYPE html>
<html>
<head><title>Pipeline Quality Report</title></head>
<body style="font-family: Arial, sans-serif; margin: 40px;">
    <h1>Pipeline Quality Report</h1>
    <p><em>Generated by Nextflow Pipeline</em></p>
    
    <h2>Pipeline Summary</h2>
    <table border="1" style="border-collapse: collapse;">
        <tr style="background-color: #f0f0f0;">
            <th>Process</th><th>Status</th><th>Description</th>
        </tr>
        <tr><td>FastQC</td><td>✅ Completed</td><td>Quality control analysis</td></tr>
        <tr><td>Trimming</td><td>✅ Completed</td><td>Adapter and quality trimming</td></tr>
        <tr><td>MultiQC</td><td>✅ Completed</td><td>Aggregated report generation</td></tr>
    </table>
    
    <h2>Results Location</h2>
    <ul>
        <li><b>FastQC Reports:</b> results/fastqc/</li>
        <li><b>Trimmed Reads:</b> results/trimmed/</li>
        <li><b>This Report:</b> results/multiqc/</li>
    </ul>
    
    <h2>Next Steps</h2>
    <p>Review individual FastQC HTML reports for detailed quality metrics of each sample.</p>
    
    <h2>Tool Information</h2>
    <p>For comprehensive MultiQC reports, install MultiQC: <code>pip install multiqc</code></p>
</body>
</html>
EOF
    fi
    
    # Ensure data directory exists
    if [ ! -d "multiqc_data" ]; then
        mkdir -p multiqc_data
        echo "Pipeline report generated on \$(date)" > multiqc_data/info.log
    fi
    
    echo "✅ Quality report generation completed"
    """
}

workflow.onComplete {
    log.info """
    ==============================================
    Pipeline Execution Complete!
    ==============================================
    Success: ${workflow.success}
    Duration: ${workflow.duration}
    Results: ${params.outdir}
    ==============================================
    """
}
'''
        
        with open(self.workflow_file, 'w') as f:
            f.write(workflow_content)
        
        print("✓ Created simplified Nextflow workflow")
    
    def _create_nextflow_config(self):
        """Create simple Nextflow configuration"""
        config_content = '''
process {
    cpus = 1
    memory = '2 GB'
    time = '30m'
}

executor {
    name = 'local'
    cpus = 4
}

report {
    enabled = true
    file = 'reports/execution_report.html'
}

timeline {
    enabled = true
    file = 'reports/timeline.html'
}
'''
        
        with open('nextflow.config', 'w') as f:
            f.write(config_content)
        
        print("✓ Created Nextflow config")
    
    def _create_sample_data(self):
        """Create sample FASTQ files with realistic quality scores"""
        import gzip
        import random
        
        print("  Creating sample data...")
        
        samples = [('sample1', 'sample1'), ('sample2', 'sample2')]
        
        for sample_id, sample_name in samples:
            for read in ['R1', 'R2']:
                filename = f"data/{sample_id}_{read}.fastq.gz"
                
                with gzip.open(filename, 'wt') as f:
                    for i in range(2000):  # Smaller files for faster processing
                        # Create realistic sequence
                        seq = ''.join(random.choices(['A', 'T', 'G', 'C'], k=100))
                        
                        # Create realistic quality scores
                        qual_scores = []
                        for pos in range(100):
                            if pos < 20:  # High quality at start
                                base_qual = random.randint(35, 40)
                            elif pos < 80:  # Good quality in middle
                                base_qual = random.randint(25, 35)
                            else:  # Lower quality at end
                                base_qual = random.randint(15, 25)
                            qual_scores.append(chr(base_qual + 33))
                        
                        qual = ''.join(qual_scores)
                        
                        f.write(f"@{sample_name}:{i+1}:1:1101:1000:2000 {read}:N:0:ATCACG\n")
                        f.write(f"{seq}\n")
                        f.write(f"+\n")
                        f.write(f"{qual}\n")
        
        print("  ✓ Created sample FASTQ files")
    
    def _create_reference_files(self):
        """Create reference genome"""
        import random
        
        print("  Creating reference genome...")
        
        with open("reference/genome.fa", 'w') as f:
            f.write(">chr1 Test chromosome\n")
            for i in range(50):
                line = ''.join(random.choices(['A', 'T', 'G', 'C'], k=100))
                f.write(line + "\n")
        
        print("  ✓ Created reference genome")
    
    def _check_fastqc_installation(self):
        """Check if FastQC is available"""
        print("  Checking FastQC installation...")
        
        try:
            result = subprocess.run(['fastqc', '--version'], 
                                  capture_output=True, text=True, timeout=10)
            if result.returncode == 0:
                print(f"  ✅ FastQC found: {result.stdout.strip()}")
            else:
                print("  ⚠️  FastQC command returned error")
        except FileNotFoundError:
            print("  ⚠️  FastQC not found - install for real analysis")
            print("     macOS: brew install fastqc")
            print("     Ubuntu: sudo apt install fastqc")
            print("     conda: conda install -c bioconda fastqc")
        except Exception as e:
            print(f"  ⚠️  Error checking FastQC: {e}")

In [ ]:
# 3. Interactive Controller
class SimpleFastQCController:
    """Simplified interactive controller"""
    
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.process = None
        self.output_thread = None
        
        # Create output widget
        self.output_widget = widgets.Output()
        
        # Status tracking
        self.current_status = "Ready"
        
    def create_control_interface(self):
        """Create simplified interactive interface"""
        
        # Control buttons
        self.run_button = widgets.Button(
            description='🚀 Run FastQC Pipeline',
            button_style='success',
            layout=widgets.Layout(width='200px')
        )
        
        self.test_button = widgets.Button(
            description='🧪 Test Mode (Quick)',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
        
        self.stop_button = widgets.Button(
            description='🛑 Stop Pipeline',
            button_style='danger',
            layout=widgets.Layout(width='150px')
        )
        
        self.clean_button = widgets.Button(
            description='🧹 Clean Output',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
        
        self.status_button = widgets.Button(
            description='📊 Check Status',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
        
        # Status display
        self.status_label = widgets.HTML(
            value=f"<b>Status:</b> {self.current_status}",
            layout=widgets.Layout(margin='10px 0px')
        )
        
        # Progress bar
        self.progress_bar = widgets.IntProgress(
            value=0, min=0, max=100,
            description='Progress:',
            layout=widgets.Layout(width='400px')
        )
        
        # Connect button events
        self.run_button.on_click(self._on_run_clicked)
        self.test_button.on_click(self._on_test_clicked)
        self.stop_button.on_click(self._on_stop_clicked)
        self.clean_button.on_click(self._on_clean_clicked)
        self.status_button.on_click(self._on_status_clicked)
        
        # Layout
        button_row1 = widgets.HBox([self.run_button, self.test_button])
        button_row2 = widgets.HBox([self.stop_button, self.clean_button, self.status_button])
        
        controls = widgets.VBox([
            widgets.HTML("<h3>🧬 Simplified FastQC Pipeline</h3>"),
            self.status_label,
            self.progress_bar,
            button_row1,
            button_row2,
            widgets.HTML("<hr>"),
            widgets.HTML("<b>Pipeline Output:</b>"),
            self.output_widget
        ])
        
        return controls
    
    def _on_run_clicked(self, button):
        """Handle run button click"""
        self._start_pipeline(test_mode=False)
    
    def _on_test_clicked(self, button):
        """Handle test button click"""
        self._start_pipeline(test_mode=True)
    
    def _start_pipeline(self, test_mode=False):
        """Start pipeline"""
        if self.process and self.process.poll() is None:
            with self.output_widget:
                print("❌ Pipeline is already running!")
            return
        
        mode = "test mode (quick)" if test_mode else "full analysis"
        self._update_status(f"Starting {mode}...")
        
        self.run_button.disabled = True
        self.test_button.disabled = True
        self.progress_bar.value = 0
        
        with self.output_widget:
            clear_output()
            print(f"🚀 Starting FastQC pipeline in {mode}...")
            print("-" * 50)
        
        # Start pipeline in thread
        try:
            self.output_thread = threading.Thread(
                target=self._run_pipeline_thread, 
                args=(test_mode,),
                daemon=True
            )
            self.output_thread.start()
        except Exception as e:
            with self.output_widget:
                print(f"❌ Error starting pipeline: {e}")
            self._update_status("❌ Start failed")
            self._enable_buttons()
    
    def _run_pipeline_thread(self, test_mode=False):
        """Run pipeline in separate thread"""
        try:
            # Check Nextflow first
            try:
                result = subprocess.run(['nextflow', '-version'], 
                                      capture_output=True, text=True, timeout=10)
                if result.returncode != 0:
                    with self.output_widget:
                        print("❌ Nextflow not working properly")
                        print("📦 Install: conda install -c bioconda nextflow")
                    self._update_status("❌ Nextflow issue")
                    return
            except FileNotFoundError:
                with self.output_widget:
                    print("❌ Nextflow not found")
                    print("📦 Install: conda install -c bioconda nextflow")
                self._update_status("❌ Nextflow missing")
                return
            
            # Build command
            cmd = [
                'nextflow', 'run', self.pipeline.workflow_file,
                '--reads', 'data/*_{R1,R2}.fastq.gz',
                '--reference', 'reference/genome.fa',
                '--outdir', 'results'
            ]
            
            if test_mode:
                cmd.extend(['--test_mode', 'true'])
            
            with self.output_widget:
                print(f"Command: {' '.join(cmd)}")
                print("")
            
            # Start process
            self.process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                universal_newlines=True,
                bufsize=1
            )
            
            # Monitor output
            mode_text = "test" if test_mode else "analysis"
            self._update_status(f"Running {mode_text}...")
            self.progress_bar.value = 10
            
            # Monitor with timeout
            start_time = time.time()
            max_time = 300 if test_mode else 1800  # 5 min test, 30 min full
            last_progress_update = start_time
            
            for line in iter(self.process.stdout.readline, ''):
                if line:
                    with self.output_widget:
                        print(line.rstrip())
                    
                    # Update progress based on output (more flexible patterns)
                    line_lower = line.lower()
                    
                    # Early progress indicators
                    if 'executor' in line_lower or 'launching' in line_lower:
                        if self.progress_bar.value < 15:
                            self.progress_bar.value = 15
                    
                    # FastQC progress
                    if 'fastqc' in line_lower:
                        if 'submitted' in line_lower or 'running' in line_lower or '[' in line:
                            if self.progress_bar.value < 30:
                                self.progress_bar.value = 30
                        elif 'completed' in line_lower or 'cached' in line_lower or 'done' in line_lower:
                            if self.progress_bar.value < 50:
                                self.progress_bar.value = 50
                    
                    # Trimmomatic progress  
                    if 'trimmomatic' in line_lower or 'trimming' in line_lower:
                        if self.progress_bar.value < 70:
                            self.progress_bar.value = 70
                    
                    # MultiQC progress
                    if 'multiqc' in line_lower or 'generating' in line_lower:
                        if self.progress_bar.value < 85:
                            self.progress_bar.value = 85
                    
                    # Completion indicators
                    if any(phrase in line_lower for phrase in [
                        'workflow completed', 'pipeline completed', 'succeeded', 
                        'execution complete', 'done', 'finished'
                    ]):
                        self.progress_bar.value = 100
                
                # Time-based progress backup (in case text patterns don't match)
                current_time = time.time()
                elapsed = current_time - start_time
                if current_time - last_progress_update > 30:  # Update every 30 seconds
                    # Gradually increase progress based on time
                    time_progress = min(85, 10 + (elapsed / max_time) * 75)
                    if time_progress > self.progress_bar.value:
                        self.progress_bar.value = int(time_progress)
                    last_progress_update = current_time
                
                # Check timeout
                if elapsed > max_time:
                    with self.output_widget:
                        print(f"\n⏰ Pipeline timed out after {max_time/60:.1f} minutes")
                    self.process.terminate()
                    self._update_status("⏰ Timed out")
                    return
            
            # Wait for completion
            self.process.wait()
            
            # Always set progress to 100% if successful, regardless of text patterns
            if self.process.returncode == 0:
                self._update_status("✅ Pipeline completed!")
                self.progress_bar.value = 100  # Force completion
                with self.output_widget:
                    print("\n🎉 Pipeline completed successfully!")
                    print("📁 Results are in the 'results/' directory")
                    if not test_mode:
                        print("📊 Open FastQC HTML reports in results/fastqc/")
            else:
                self._update_status("❌ Pipeline failed")
                with self.output_widget:
                    print(f"\n❌ Pipeline failed (exit code {self.process.returncode})")
                    
        except Exception as e:
            self._update_status(f"❌ Error: {str(e)[:30]}...")
            with self.output_widget:
                print(f"❌ Pipeline error: {e}")
                
        finally:
            self._enable_buttons()
    
    def _on_stop_clicked(self, button):
        """Handle stop button click"""
        if self.process and self.process.poll() is None:
            try:
                self.process.terminate()
                self._update_status("🛑 Stopped")
                with self.output_widget:
                    print("\n🛑 Pipeline stopped by user")
            except Exception as e:
                with self.output_widget:
                    print(f"Error stopping pipeline: {e}")
        else:
            with self.output_widget:
                print("No pipeline is currently running")
    
    def _on_clean_clicked(self, button):
        """Handle clean button click"""
        with self.output_widget:
            print("🧹 Cleaning output directories...")
        
        try:
            import shutil
            
            dirs_to_clean = ['results', 'work', '.nextflow']
            for directory in dirs_to_clean:
                if Path(directory).exists():
                    shutil.rmtree(directory)
                    with self.output_widget:
                        print(f"  ✓ Removed {directory}/")
            
            Path('results').mkdir(exist_ok=True)
            self._update_status("🧹 Cleaned")
            self.progress_bar.value = 0
            
            with self.output_widget:
                print("✅ Cleanup completed!")
                
        except Exception as e:
            with self.output_widget:
                print(f"❌ Cleanup error: {e}")
    
    def _on_status_clicked(self, button):
        """Handle status button click"""
        with self.output_widget:
            print("\n📊 System Status Check")
            print("-" * 30)
        
        # Check Nextflow
        try:
            result = subprocess.run(['nextflow', '-version'], 
                                  capture_output=True, text=True, timeout=10)
            with self.output_widget:
                print(f"✅ Nextflow: {result.stdout.strip()}")
        except FileNotFoundError:
            with self.output_widget:
                print("❌ Nextflow: Not found")
        except Exception as e:
            with self.output_widget:
                print(f"❌ Nextflow: Error - {e}")
        
        # Check FastQC
        try:
            result = subprocess.run(['fastqc', '--version'], 
                                  capture_output=True, text=True, timeout=10)
            with self.output_widget:
                print(f"✅ FastQC: {result.stdout.strip()}")
        except FileNotFoundError:
            with self.output_widget:
                print("❌ FastQC: Not found (will use test mode)")
        except Exception as e:
            with self.output_widget:
                print(f"❌ FastQC: Error - {e}")
        
        # Check input files
        fastq_files = list(Path("data").glob("*.fastq.gz"))
        with self.output_widget:
            print(f"\n📄 Input files: {len(fastq_files)} FASTQ files found")
        
        # Check results
        if Path("results").exists():
            fastqc_reports = list(Path("results").rglob("*.html"))
            with self.output_widget:
                print(f"📊 Results: {len(fastqc_reports)} HTML reports found")
        else:
            with self.output_widget:
                print("📊 Results: No results directory")
    
    def _update_status(self, status):
        """Update status display"""
        self.current_status = status
        self.status_label.value = f"<b>Status:</b> {status}"
    
    def _enable_buttons(self):
        """Re-enable control buttons"""
        self.run_button.disabled = False
        self.test_button.disabled = False

In [ ]:
# 4. Initialize Display
# Create pipeline instance
print("🔧 Initializing simplified pipeline...")
pipeline = SimpleFastQCPipeline()

# Setup pipeline
pipeline.setup_pipeline()

# Create controller
print("🎮 Creating controls...")
controller = SimpleFastQCController(pipeline)

# Create and display interface
control_interface = controller.create_control_interface()

print("\n✅ Simplified setup complete!")

# Display the interface
display(control_interface)

In [ ]:
# 5. Utiliy Functions
def analyze_results():
    """Analyze pipeline results"""
    print("📈 Analyzing Pipeline Results")
    print("=" * 35)
    
    if not Path("results").exists():
        print("❌ No results directory found. Run the pipeline first!")
        return
    
    # Check FastQC results
    fastqc_dir = Path("results/fastqc")
    if fastqc_dir.exists():
        html_files = list(fastqc_dir.glob("*.html"))
        zip_files = list(fastqc_dir.glob("*.zip"))
        
        print(f"📊 FastQC Results:")
        print(f"   📄 HTML reports: {len(html_files)}")
        for f in html_files:
            print(f"      - {f.name}")
        print(f"   📦 Data files: {len(zip_files)}")
    else:
        print("📊 No FastQC results found")
    
    # Check MultiQC results
    multiqc_dir = Path("results/multiqc")
    if multiqc_dir.exists():
        multiqc_files = list(multiqc_dir.glob("*.html"))
        print(f"\n📋 MultiQC Results: {len(multiqc_files)} reports")
        for f in multiqc_files:
            print(f"   📄 {f.name}")
            print(f"      Open: file://{f.absolute()}")
    
    # Check trimmed files
    trimmed_dir = Path("results/trimmed")
    if trimmed_dir.exists():
        trimmed_files = list(trimmed_dir.glob("*.fastq.gz"))
        print(f"\n✂️ Trimmed Files: {len(trimmed_files)} files")
    
    print(f"\n🎉 Analysis complete! Found results in {len(list(Path('results').iterdir()))} subdirectories.")

def view_sample_report():
    """Show a sample FastQC report if available"""
    fastqc_dir = Path("results/fastqc")
    if not fastqc_dir.exists():
        print("❌ No FastQC results found. Run the pipeline first!")
        return
    
    html_files = list(fastqc_dir.glob("*.html"))
    if not html_files:
        print("❌ No HTML reports found.")
        return
    
    sample_file = html_files[0]
    print(f"📄 Sample FastQC Report: {sample_file.name}")
    print(f"🔗 Open in browser: file://{sample_file.absolute()}")
    
    # Try to show first few lines of the report
    try:
        with open(sample_file, 'r') as f:
            content = f.read()
            if 'Test Mode' in content:
                print("📋 Type: Test mode report")
            elif 'FastQC' in content and 'Basic Statistics' in content:
                print("📋 Type: Real FastQC analysis report")
            else:
                print("📋 Type: Installation required notice")
    except Exception as e:
        print(f"❌ Error reading report: {e}")

# Make functions available
print("\n🛠️ Utility functions available:")
print("• analyze_results() - Analyze all pipeline outputs")
print("• view_sample_report() - Show info about generated reports")

print("\n" + "="*60)
print("🎉 SIMPLIFIED FASTQC PIPELINE READY!")
print("="*60)
print("✅ Uses system FastQC installation (no conda complexity)")
print("✅ Test mode for quick validation")
print("✅ Simplified interface with essential controls")
print("✅ Real FastQC analysis when FastQC is installed")
print("\n👆 Choose your mode:")
print("   • Test Mode - Quick validation (always works)")
print("   • Full Mode - Real analysis (requires FastQC installation)")

In [ ]:
## End of Notebook ##